# Chapter 12: Error Handling and Remediation

Key Takeaways:
- **Robustness via Fallbacks**: Design systems that gracefully handle failure by delegating to specialized fallback agents.
- **State-Aware Remediation**: Agents can check the current state (e.g., "did the previous step fail?") to decide whether to act.
- **Sequential Orchestration**: Use a sequential flow to ensure the primary attempt happens before any remediation steps.
- **Clear User Communication**: A final agent ensures the user receives a coherent response regardless of which underlying path was taken.

### Heuristic: *Plan for failure with dedicated fallback loops.*

## Setup and Initialization

In [13]:
import os
import sys
from dotenv import load_dotenv
from langchain_core.tools import tool

# Add scripts directory to path to import custom modules
PROJECT_ROOT = os.path.dirname(os.getcwd())
SCRIPTS_DIR = os.path.join(PROJECT_ROOT, "scripts")
sys.path.insert(0, SCRIPTS_DIR)

# Import our custom ADK implementation
from custom_adk import Agent, SequentialAgent

# Load environment variables
load_dotenv()

# Verify API key
if not os.getenv("GOOGLE_API_KEY"):
    print("❌ Please set the GOOGLE_API_KEY environment variable.")
else:
    print("✅ Configuration Loaded")

✅ Configuration Loaded


## 1. Define Tools

We will define two tools: one that is precise but "flaky" (simulated failure), and one that is general but reliable.

In [14]:
@tool
def get_precise_location_info(address: str) -> str:
    """Get precise coordinates for a specific address. Returns None if not found."""
    # Simulate failure for a specific address to demonstrate fallback
    if "Ambiguous St" in address:
        print(f"   [Tool] get_precise_location_info failed for: {address}")
        return None
    
    print(f"   [Tool] get_precise_location_info successful for: {address}")
    return "Lat: 40.7128, Long: -74.0060"

@tool
def get_general_area_info(city: str) -> str:
    """Get general area information for a city."""
    print(f"   [Tool] get_general_area_info called for: {city}")
    return "New York City Area: High density, urban environment."

## 2. Define Agents

We create three agents:
1. **Primary Handler**: Tries the precise tool.
2. **Fallback Handler**: Checks if the primary failed, and if so, uses the general tool.
3. **Response Agent**: Formulates the final answer for the user.

In [15]:
# Agent 1: Tries the primary tool. Its focus is narrow and clear.
primary_handler = Agent(
    name="primary_handler",
    model="gemini-2.5-flash",
    instruction="""
Your job is to get precise location information.
Use the get_precise_location_info tool with the user's provided address.
""",
    tools=[get_precise_location_info]
)

# Agent 2: Acts as the fallback handler, checking state to decide its action.
fallback_handler = Agent(
    name="fallback_handler",
    model="gemini-2.5-flash",
    instruction="""
Check if the primary location lookup failed by looking at state["primary_location_failed"].
- If it is True, extract the city from the user's original query and use the get_general_area_info tool.
- If it is False, do nothing.
""",
    tools=[get_general_area_info]
)

# Agent 3: Presents the final result from the state.
response_agent = Agent(
    name="response_agent",
    model="gemini-2.5-flash",
    instruction="""
Review the location information stored in state["location_result"].
Present this information clearly and concisely to the user.
If state["location_result"] does not exist or is empty, apologize that you could not retrieve the location.
""",
    tools=[] # This agent only reasons over the final state.
)

## 3. Orchestration

We use a `SequentialAgent` to run these agents in a guaranteed order.

In [16]:
# The SequentialAgent ensures the handlers run in a guaranteed order.
robust_location_agent = SequentialAgent(
    name="robust_location_agent",
    sub_agents=[primary_handler, fallback_handler, response_agent]
)

## 4. Execution Examples

### Scenario 1: Success Path
The address is valid, so the primary handler succeeds.

In [17]:
print("🟢 --- RUNNING SUCCESS SCENARIO ---")
initial_state = {
    "user_query": "123 Main St, New York, NY",
    "primary_location_failed": False, # Default assumption
    "location_result": None
}

final_state = robust_location_agent.run(initial_state)

🟢 --- RUNNING SUCCESS SCENARIO ---
Starting Sequential Workflow: robust_location_agent

[Agent: primary_handler] Running...
[Agent: primary_handler] Response: 
[Agent: primary_handler] Calling tool: get_precise_location_info with {'address': '123 Main St, New York, NY'}
   [Tool] get_precise_location_info successful for: 123 Main St, New York, NY
[Agent: primary_handler] Tool Result: Lat: 40.7128, Long: -74.0060

[Agent: fallback_handler] Running...
[Agent: fallback_handler] Response: 

[Agent: response_agent] Running...
[Agent: response_agent] Response: The location information retrieved is: Lat: 40.7128, Long: -74.0060


### Scenario 2: Failure & Fallback Path
The address is ambiguous, causing the primary handler to fail. The fallback handler should kick in.

In [18]:
print("\n🔴 --- RUNNING FAILURE/FALLBACK SCENARIO ---")
initial_state = {
    "user_query": "404 Ambiguous St, New York, NY",
    "primary_location_failed": False,
    "location_result": None
}

final_state_fallback = robust_location_agent.run(initial_state)


🔴 --- RUNNING FAILURE/FALLBACK SCENARIO ---
Starting Sequential Workflow: robust_location_agent

[Agent: primary_handler] Running...
[Agent: primary_handler] Response: 
[Agent: primary_handler] Calling tool: get_precise_location_info with {'address': '404 Ambiguous St, New York, NY'}
   [Tool] get_precise_location_info failed for: 404 Ambiguous St, New York, NY
[Agent: primary_handler] Tool Result: None

[Agent: fallback_handler] Running...
[Agent: fallback_handler] Response: 
[Agent: fallback_handler] Calling tool: get_general_area_info with {'city': 'New York'}
   [Tool] get_general_area_info called for: New York
[Agent: fallback_handler] Tool Result: New York City Area: High density, urban environment.

[Agent: response_agent] Running...
[Agent: response_agent] Response: The location information retrieved is: New York City Area: High density, urban environment.


## Conclusion

This pattern demonstrates how to build resilience into agentic workflows. Instead of failing hard when a tool doesn't work, the system intelligently reroutes to a fallback strategy, ensuring the user still receives useful information.